In [1]:
import numpy as np
import pandas as pd

### FM，对特征进行组合

　　FM（Factorization Machine），用于解决稀疏数据下的特征组合问题，常用于广告的CTR预估。工业界用的较多的是基于LR（logistic回归）的点击预估策略，使用GBDT+FM用树和FM来做高阶特征生成，最后在使用sigmoid（套用logistic回归，将其映射为概率），能增强LR的非线性能力。与DNN（深度神经网络）用线性组合+非线性函数来做高阶特征生成相比，二者效果相差不大。

　　某些特征经过关联之后，与label之间的相关性就会提高。例如，“USA”与“Thanksgiving”、“China”与“Chinese New Year”这样的关联特征，对用户的点击有着正向的影响。换句话说，来自“China”的用户很可能会在“Chinese New Year”有大量的浏览、购买行为，而在“Thanksgiving”却不会有特别的消费行为。这种关联特征与label的正向相关性在实际问题中是普遍存在的，如“化妆品”类商品与“女”性，“球类运动配件”的商品与“男”性，“电影票”的商品与“电影”品类偏好等。因此，引入两个特征的组合是非常有意义的。

### 使用one-hot编码的问题

　　下面以一个示例引入FM模型。假设一个广告分类的问题，根据用户和广告位相关的特征，预测用户是否点击了广告。源数据如下：

In [4]:
ad_df = pd.DataFrame(columns=['Clicked', 'Country', 'Day', 'Ad_type'])
ad_df['Clicked'] = [1, 0, 1]
ad_df['Country'] = ['USA', 'China', 'China']
ad_df['Day'] = ['26/11/15', '1/7/14', '19/2/15']
ad_df['Ad_type'] = ['Movie', 'Game', 'Game']

In [5]:
ad_df

,Clicked,Country,Day,Ad_type
0,1,USA,26/11/15,Movie
1,0,China,1/7/14,Game
2,1,China,19/2/15,Game


　　"Clicked"是label，Country、Day、Ad_type是特征。由于三种特征都是categorical类型的，需要经过独热编码（One-Hot Encoding）转换成数值型特征。

In [8]:
pd.get_dummies(ad_df)

,Clicked,Country_China,Country_USA,Day_1/7/14,Day_19/2/15,Day_26/11/15,Ad_type_Game,Ad_type_Movie
0,1,0,1,0,0,1,0,1
1,0,1,0,1,0,0,1,0
2,1,1,0,0,1,0,1,0


　　由上表可以看出，经过One-Hot编码之后，大部分样本数据特征是比较稀疏的。上面的样例中，每个样本有7维特征，但平均仅有3维特征具有非零值。实际上，这种情况并不是此例独有的，在真实应用场景中这种情况普遍存在。例如，CTR/CVR预测时，用户的性别、职业、教育水平、品类偏好，商品的品类等，经过One-Hot编码转换后都会导致样本数据的稀疏性。特别是商品品类这种类型的特征，如商品的末级品类约有550个，采用One-Hot编码生成550个数值特征，但每个样本的这550个特征，有且仅有一个是有效的（非零）。由此可见，数据稀疏性是实际问题中不可避免的挑战。<br >
　　One-Hot编码的另一个特点就是导致特征空间大。例如，淘宝上的商品有上百万，使用one-hot编码后，一个物品ID项可扩展上百万的维度，由此特征空间剧增带来的数据的稀疏性。

### FM公式推导
　　这里以二阶多项式模型为例，因其包含组合特征$x_{i} x_{j}$，公式如下：
$$y(x) = w_{0} + \sum _{i=1}^{n} w_{i}x_{i} + \sum _{i=1}^{n} \sum _{j=i+1}^{n} w_{ij} x_{i}x_{j}$$

　　其中，n代表样本的特征数量，$x_{i}$是第i个特征，$w_{0}$、$w_{i}$、$w_{ij}$是模型参数。

$$\sum _{i=1}^{n-1} \sum _{j=i+1}^{n} < v_{i}, v_{j}> x_{i}x_{j}\\
= \frac{1}{2} \sum _{i=1}^{n} \sum _{j=1}^{n} < v_{i}, v_{j}> x_{i}x_{j} - \frac{1}{2} \sum _{i=1}^{n} < v_{i}, v_{i}> x_{i}x_{i}\\
=\frac{1}{2} (\sum _{i=1}^{n} \sum _{j=1}^{n} \sum _{k=1}^{K} v_{ik} v_{jk} x_{i}x_{j} - \sum _{i=1}^{n} \sum _{k=1}^{K}v_{ik}v_{ik}  x_{i}x_{i})\\
=\frac{1}{2} \sum _{k=1}^{K} ((\sum _{i=1}^{n} v_{ik} x_{i}) - \sum _{i=1}^{n} v_{ik}^{2} x_{i}^{2})$$